# [Machine Learning Compilation](https://mlc.ai/chapter_introduction/index.html)

This notebook relates to the corresponding course.

* The first half of the notebook closely corresponds to [2.4. TensorIR: Tensor Program Abstraction Case Study](https://mlc.ai/chapter_tensor_program/case_study.html).
* The second half is a free interpretation of the quest __how to optimize an operator.__

In [1]:
import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T

In [2]:
dtype = "float32"
a_np = np.random.rand(128, 128).astype(dtype)
b_np = np.random.rand(128, 128).astype(dtype)
# a @ b is equivalent to np.matmul(a, b)
c_mm_relu = np.maximum(a_np @ b_np, 0)

In [3]:
a_nd = tvm.nd.array(a_np)
b_nd = tvm.nd.array(b_np)
c_nd = tvm.nd.empty((128, 128), dtype="float32")
type(c_nd)

tvm.runtime.ndarray.NDArray

In [4]:
@tvm.script.ir_module
class MyModule:
    @T.prim_func
    def mm_relu(A: T.Buffer((128, 128), "float32"),
                B: T.Buffer((128, 128), "float32"),
                C: T.Buffer((128, 128), "float32")):
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        Y = T.alloc_buffer((128, 128), dtype="float32")
        for i, j, k in T.grid(128, 128, 128):
            with T.block("Y"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j)
                vk = T.axis.reduce(128, k)
                with T.init():
                    Y[vi, vj] = T.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in T.grid(128, 128):
            with T.block("C"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j)
                C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

In [5]:
MyModule.show()

## Exercise

Let's modify the way we loop through the structure.
`jfactor` determines how to balance between `j0` and `j1`.

In [6]:
def transform(mod, jfactor):
    sch = tvm.tir.Schedule(mod)
    block_Y = sch.get_block("Y", func_name="mm_relu")
    i, j, k = sch.get_loops(block_Y)
    j0, j1 = sch.split(j, factors=[None, jfactor])
    sch.reorder(j0, k, j1)
    block_C = sch.get_block("C", "mm_relu")
    sch.reverse_compute_at(block_C, j0)
    return sch.mod

`check_identity` makes sure we always adhere to the result of the reference.

In [7]:
def check_identity(rt_lib):
   #rt_lib_after = tvm.build(mod, target="llvm")
   rt_lib["mm_relu"](a_nd, b_nd, c_nd)
   np.testing.assert_allclose(c_mm_relu, c_nd.numpy(), rtol=1e-5)
   

For each configuration, measure the runtime.

In [20]:
def benchmark(jf, performance):
    mod_transformed = transform(MyModule, jfactor=jf)
    rt_lib_transformed = tvm.build(mod_transformed, "llvm")
    #check_identity(mod_transformed)
    check_identity(rt_lib_transformed)

    f_timer_transformed = rt_lib_transformed.time_evaluator("mm_relu", tvm.cpu())
    t = f_timer_transformed(a_nd, b_nd, c_nd).mean
    #print("Time cost of transformed for jfactor=={}: {} sec".format(jf, t))
    performance[jf] = (t, mod_transformed)
    return performance


In [21]:
performance = {}
for i in range(1, 128, 1):
   performance = benchmark(i, performance)

In [26]:
import operator
sorted_dict = dict(sorted(performance.items(), key=operator.itemgetter(1)))

for k, v in sorted_dict.items():
   print("jfactor: {}, t: {}s".format(k, v[0]))

jfactor: 32, t: 0.00015303000000000002s
jfactor: 12, t: 0.00020095s
jfactor: 64, t: 0.00022651s
jfactor: 10, t: 0.00023535s
jfactor: 8, t: 0.00023564s
jfactor: 13, t: 0.00030172s
jfactor: 4, t: 0.00038056s
jfactor: 15, t: 0.00039287999999999997s
jfactor: 16, t: 0.0004205199999999999s
jfactor: 20, t: 0.00042167s
jfactor: 14, t: 0.00042214s
jfactor: 18, t: 0.00042917s
jfactor: 19, t: 0.00044241s
jfactor: 11, t: 0.00045303s
jfactor: 17, t: 0.00053091s
jfactor: 6, t: 0.00054278s
jfactor: 3, t: 0.0006664799999999999s
jfactor: 7, t: 0.00067649s
jfactor: 9, t: 0.00071099s
jfactor: 5, t: 0.00071331s
jfactor: 2, t: 0.00090735s
jfactor: 35, t: 0.00109172s
jfactor: 48, t: 0.00112165s
jfactor: 34, t: 0.00114607s
jfactor: 46, t: 0.00114631s
jfactor: 36, t: 0.0011612399999999998s
jfactor: 21, t: 0.0011694000000000001s
jfactor: 33, t: 0.0011766699999999999s
jfactor: 37, t: 0.00119457s
jfactor: 44, t: 0.00121251s
jfactor: 68, t: 0.00121856s
jfactor: 38, t: 0.0012265700000000002s
jfactor: 50, t: 0.0012

In [37]:
#print(performance)
# Using min() + list comprehension + values()
# Finding min value keys in dictionary
#vals = min(performance.values())
times= [x[0] for x in performance.values()]
min_t= min(times)
res = [key for key in performance if performance[key][0] == min_t]
 
# printing result 
print("Keys with minimum values are: jfactor=={} which takes {}s to compute.".format(str(res[0]), min_t))

Keys with minimum values are: jfactor==32 which takes 0.00015303000000000002s to compute.


In [43]:
print("The winner in terms of compute (less is more):")
(performance[res[0]][1]).show()

The winner in terms of compute (less is more):


## Conclusion

* __TensorIR__ helps to model operators. It also helps to optimize the loop structure (schedule) of these operators.
* Optimization is done relative to the underlying hardware. Optimization probably depends on things like cache size(s), tensor dimensions.